In [124]:
import tensorflow_datasets as tfds
ds = tfds.load('wikipedia/20201201.es')

In [188]:
numArticles = 10000

In [176]:
df = tfds.as_dataframe(ds['train'].take(numArticles))

In [126]:
df.head(102)

,text,title
0,b'REDIRECCI\xc3\x93N Jacobaea vulgaris',b'Usai gaistoco besatbat'
1,b'La casa Botines o casa Fern\xc3\xa1ndez y An...,b'Casa Botines'
2,b'El Westland Lysander (Lisandro) fue un aerop...,b'Westland Lysander'
3,b'REDIRECCI\xc3\x93N Iluminados de Baviera.',b'Illuminatus'
4,b'REDIRECCI\xc3\x93N Sergio de Lis',b'Sergio de lis'
...,...,...
97,b'El \xc3\x81rea metropolitana de la Gran Barc...,b'Gran Barcelona'
98,"b'Mierko Blazina (Gorizia, Friuli-Venecia Juli...",b'Mierko Blazina'
99,b'REDIRECCI\xc3\x93N Galium palustre',b'Galio palustre'
100,b'El t\xc3\xa9rmino observatorio puede referir...,b'Observatorio (desambiguaci\xc3\xb3n)'


In [182]:
def remove_punctuation(texto):
  return "".join(c for c in texto if c.isalnum() or c == ' ')  

tilde_table = str.maketrans('áéíóúÁÉÍÓÚäëïöüÄËÏÖÜ','aeiouAEIOUaeiouAEIOU', '´')
def eliminar_tilde(texto):
  return texto.translate(tilde_table)

def clean_text(texto):
  return remove_punctuation(eliminar_tilde(texto))

In [191]:
url = 'https://en.wikipedia.org/w/api.php'
params = dict(
    action='query',
    titles='',
    prop='links',
    format='json'
)
directedGraphTxt = open("directedGraph.txt", "w")

idx = 0

for row in df.iterrows():
  if (idx % 100):
    print('processing '+str(idx)+' article...')
    print(str(idx/numArticles*100)+'%')
  idx += 1  


  value = row[1]
  artIdx = row[0]
  title = value['title'].decode('utf-8')
  content = value['text'].decode('utf-8')
  params['titles'] = title
  resp = requests.get(url=url, params=params)
  data = resp.json()
  links_to = []
  for key, value in data['query']['pages'].items():
    if (key == '-1'):
      continue
    links = value['links']
    for link in links:
      newTitle = clean_text(link['title'])
      if (len(newTitle) == 0):
        continue
      links_to.append(newTitle)

  title = clean_text(title)
  if (len(title) == 0):
    continue
  
  articleTxt = open("./es_articles/"+title, "w")
  articleTxt.write(content)
  articleTxt.close()

  if (len(links_to) == 0):
    directedGraphTxt.write(title+"\t"+":"+str(1/numArticles)+"\n")
  else:
    directedGraphTxt.write(title+"\t"+','.join(links_to)+":"+str(1/numArticles)+"\n")

directedGraphTxt.close()

Streaming output truncated to the last 5000 lines.
processing 7475 article...
74.75%
processing 7476 article...
74.76%
processing 7477 article...
74.77000000000001%
processing 7478 article...
74.78%
processing 7479 article...
74.79%
processing 7480 article...
74.8%
processing 7481 article...
74.81%
processing 7482 article...
74.82%
processing 7483 article...
74.83%
processing 7484 article...
74.83999999999999%
processing 7485 article...
74.85000000000001%
processing 7486 article...
74.86%
processing 7487 article...
74.87%
processing 7488 article...
74.88%
processing 7489 article...
74.89%
processing 7490 article...
74.9%
processing 7491 article...
74.91%
processing 7492 article...
74.92%
processing 7493 article...
74.92999999999999%
processing 7494 article...
74.94%
processing 7495 article...
74.95%
processing 7496 article...
74.96000000000001%
processing 7497 article...
74.97%
processing 7498 article...
74.98%
processing 7499 article...
74.99%
processing 7501 article...
75.01%
process

In [ ]:
!zip -r es_articles.zip es_articles

In [185]:
!rm -r es_articles

In [186]:
!mkdir es_articles